In [1]:
import re
from typing import *
import numpy as np
import scipy as sp
import mne
from sklearn.ensemble import RandomForestClassifier
from typing import List, Tuple
from nodestimation.learning.selection import collect_statistic, \
    compute_importance, collect_cross_statistic, make_selection_map, \
    select, separate_datasets, selected_statistic, choose_best, selected_data, make_feature_selection
from nodestimation.project import  find_subject_dir
from nodestimation.pipeline import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import nilearn.plotting as nplt
from nodestimation.project.actions import read
import nodestimation as nd
from nodestimation.project.subject import Subject



subjects = pipeline(methods=['wpli', 'psd'],
                    freq_bands=[(0.5, 4), (4, 7), (7, 14), (14, 30), (30, 70)])

All computation has been already done, loading of the existing file with the solution...


In [2]:
for subject in subjects:
    print(subject.nodes[0].center_coordinates)

[-10.78637064  42.63390845   1.74274268]
[-9.71805658 36.7578724   7.65078827]
[-8.75405794 37.20337942  9.71210528]
[-10.45833878  36.08684447  11.17185009]
[-10.09544819  36.75362593  11.15145921]
[-9.94694863 39.24909556 11.76806899]
[-10.45159575  43.03892287   2.59733891]
[-9.966018   38.18533662 11.00012051]
[-9.32307655 36.58529126  9.09617167]


In [19]:
class A(object):
    def __init__(self,num1:int):
        self.num1 = num1
    def __str__(self):
        return 'A: {}'.format(self.num1)
    @property
    def num1(self):
        return self
    @num1.setter
    def num1(self, value):
      self._num1 = value
    @num1.getter
    def num1(self):
        return self._num1
    def add1(self):
        self.num1 += 1


a = [A(1), A(2), A(3), A(4)]
print('\n'.join(dir(object)))

__class__
__delattr__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__


In [ ]:
titanic = pd.read_csv('/home/user/Downloads/titanic.csv')

titanic['Survived'] = pd.Series(surv == 1 for surv in titanic['Survived'])
titanic['Sex'] = pd.Series([int(sex == 'female') for sex in titanic["Sex"]])
for row in titanic.index:
    if not np.isfinite(titanic.loc[row]['Age']):
        titanic.loc[row, 'Age'] = titanic['Age'].mean()
titanic['Embarked'] = pd.Series({'S': 0, 'C': 1, 'Q': 2}[emb] for emb in titanic['Embarked'])
dataset = titanic.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis=1)
data = dataset.drop(['Survived'], axis=1)
print(data)
target = dataset['Survived']
survived = pd.DataFrame([data.iloc[i] for i in range(dataset.shape[0]) if dataset.iloc[i]['Survived']])
died = pd.DataFrame([data.iloc[i] for i in range(dataset.shape[0]) if not dataset.iloc[i]['Survived']])

statistic = collect_statistic(data)

print(compute_importance(survived, statistic))
print(compute_importance(died, statistic))

In [ ]:
rforest = RandomForestClassifier()
rforest.fit(data, target)
importances_df = pd.DataFrame(rforest.feature_importances_, columns=['Feature_Importance'],
                              index=data.columns)
print(importances_df)

In [ ]:
sp1 = 0
dp1 = 0
sp2 = 0
dp2 = 0
sp3 = 0
dp3 = 0
for passenger in dataset.index:
    if dataset.loc[passenger]['Survived'] == 1:
        if dataset.loc[passenger]['Pclass'] == 1:
            sp1 += 1
        elif dataset.loc[passenger]['Pclass'] == 2:
            sp2 += 1
        elif dataset.loc[passenger]['Pclass'] == 3:
            sp3 += 1
    elif dataset.loc[passenger]['Survived'] == 0:
        if dataset.loc[passenger]['Pclass'] == 1:
            dp1 += 1
        elif dataset.loc[passenger]['Pclass'] == 2:
            dp2 += 1
        elif dataset.loc[passenger]['Pclass'] == 3:
            dp3 += 1

print('Pclass1: ', sp1, ': ', dp1)
print('Pclass2: ', sp2, ': ', dp2)
print('Pclass3: ', sp3, ': ', dp3)

In [ ]:
csv1 = pd.read_csv('/home/user/Downloads/B1C2_node_estimation_pipeline_file_prepare_data_output_dataset.csv')
csv2 = pd.read_csv('/home/user/Downloads/B1R1_node_estimation_pipeline_file_prepare_data_output_dataset.csv')
csv3 = pd.read_csv('/home/user/Downloads/H2S1_node_estimation_pipeline_file_prepare_data_output_dataset.csv')

datasets = [csv1, csv2, csv3]

best_data = make_feature_selection(datasets, 'resected')
common = pd.concat([best_sample for best_sample in best_data], axis=0)
print(best_data[0].columns)

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(a['B1C2']['X'], a['B1C2']['Y'])
# from sklearn.svm import SVR
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
#
# regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
# regr.fit(x_train, y_train)
# y_pred = regr.predict(x_test)
#
# print(y_pred)
# print()
# print(y_test)

In [ ]:
# from nodestimation.mlearning.features import prepare_connectivity, prepare_data
# con = pkl.load(open(subjects[0].data['con'], 'rb'))

subjects_dir, subjects_ = find_subject_dir()
labels = mne.read_labels_from_annot('B1C2', parc='aparc', subjects_dir=subjects_dir)
label_names = [label.name for label in labels]
lh_labels = [name for name in label_names if name.endswith('lh')]
rh_labels = [name for name in label_names if name.endswith('rh')]

label_ypos_lh = list()

for name in lh_labels:
    idx = label_names.index(name)
    ypos = np.mean(labels[idx].pos[:, 1])
    label_ypos_lh.append(ypos)

try:
    idx = label_names.index('Brain-Stem')

except ValueError:
    pass

else:
    ypos = np.mean(labels[idx].pos[:, 1])
    lh_labels.append('Brain-Stem')
    label_ypos_lh.append(ypos)


lh_labels = [label for (yp, label) in sorted(zip(label_ypos_lh, lh_labels))]

rh_labels = [label[:-2] + 'rh' for label in lh_labels
             if label != 'Brain-Stem' and label[:-2] + 'rh' in rh_labels]


node_colors = [label.color for label in labels]

node_order = lh_labels[::-1] + rh_labels

node_angles = mne.viz.circular_layout(label_names, node_order, start_pos=90,
                              group_boundaries=[0, len(label_names) // 2])

In [ ]:
methods = [
    'coh',
    'imcoh',
    'plv',
    'ciplv',
    'ppc',
    'pli'
]

freq_bands = [
    '0.5-4Hz',
    '4-7Hz',
    '7-14Hz',
    '14-30Hz'
]

for method in methods:
    for freq_band in freq_bands:
        fig = plt.figure(num=None, figsize=(25, 25), facecolor='black')
        mne.viz.plot_connectivity_circle(con[freq_band][method]['con'][:, :, 0], label_names, n_lines=300,
                                         node_angles=node_angles, node_colors=node_colors,
                                         title='All-to-All Connectivity Epilepsy Condition ({} for {})'
                                         .format(method, freq_band), padding=8, fontsize_title=35, fontsize_colorbar=25,
                                         fontsize_names=20, fig=fig
                                         )

In [ ]:
for subject in subjects:
    fig, ax = plt.subplots(figsize=(15,15))
    display = nplt.plot_glass_brain(None, display_mode='lyrz', figure=fig, axes=ax)
    spared = [node.nilearn_coordinates for node in subject.nodes if node.type == 'spared']
    resected = [node.nilearn_coordinates for node in subject.nodes if node.type == 'resected']
    resection = read['resec-mni'](subject.data['resec-mni'])
    display.add_markers(resection, marker_color="violet", marker_size=1)
    display.add_markers(np.array(spared), marker_color="yellow", marker_size=100)
    display.add_markers(np.array(resected), marker_color="red", marker_size=250)


